In [10]:
        # Importing libraries 
        
        import numpy as np
        import argparse
        import matplotlib.pyplot as plt
        import cv2
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense, Dropout, Flatten
        from tensorflow.keras.layers import Conv2D
        from tensorflow.keras.optimizers import Adam
        from tensorflow.keras.layers import MaxPooling2D
        from tensorflow.keras.preprocessing.image import ImageDataGenerator
        import os
        from keras.models import model_from_json


        # generating testing and training data
        # download the zip file from kaggle "facial emotion zip file"

        train_dir = 'C:\data\emotions/train'
        val_dir = 'C:\data\emotions/test'
        num_train = 28709
        num_val = 7178
        batch_size = 64 # try changing the batch size to improve accuracy
        num_epoch = 50  # try changing the number of epochs to improve accuracy

        train_datagen = ImageDataGenerator(rescale=1./255)
        val_datagen = ImageDataGenerator(rescale=1./255)
        train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(48,48),
                batch_size=batch_size,
                color_mode="grayscale",
                class_mode='categorical')
        validation_generator = val_datagen.flow_from_directory(
                val_dir,
                target_size=(48,48),
                batch_size=batch_size,
                color_mode="grayscale",
                class_mode='categorical')

        # Creating the neural network


        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())

        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(0.5))
        
        model.add(Dense(7, activation='softmax'))


    # change the mode to "train" to train the neural network and then the model would be saved as model.h5
    # change the mode to "display" to load the weights and give prediction on live feed
        mode = "display"
        # If you want to train the same model or try other models, go for this


        if mode == "train":
            model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
            model_info = model.fit_generator(
                    train_generator,
                    steps_per_epoch=num_train // batch_size,
                    epochs=num_epoch,
                    validation_data=validation_generator,
                    validation_steps=num_val // batch_size)
            
            model_json = model.to_json()
            with open("model.json", "w") as json_file:
                json_file.write(model_json)
            model.save_weights("model.h5")
            print("Saved model to disk")



        # emotions will be displayed on your face from the webcam feed
        elif mode == "display":
            model.load_weights('model.h5') # loading the weights by the training of above neural network
            # prevents openCL usage and unnecessary logging messages
            cv2.ocl.setUseOpenCL(False)
            # dictionary which assigns each label an emotion (alphabetical order)
            emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
            # starting the webcam feed
            cap = cv2.VideoCapture(0)
            while True:
                # detecting the face and creating a rectangular box on faces
                ret, frame = cap.read()
                if not ret:
                    break
                facecasc = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)
                for (x, y, w, h) in faces:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
                    roi_gray = gray[y:y+h, x:x+w] 
                    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
                    prediction = model.predict(cropped_img)
                    maxindex = int(np.argmax(prediction))
                    cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.imshow('Video', cv2.resize(frame,(1200,960),interpolation = cv2.INTER_CUBIC))
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
